# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [56]:
import fcwt

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [10]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

In [11]:
def update_array_by_mask(array, mask, value=np.nan):
    """
    Update elements of an array based on a mask and replace them with a specified value.

    Parameters:
    - array (np.array): The input numpy array whose values are to be updated.
    - mask (np.array): A boolean array with the same shape as `array`. Elements of `array` corresponding to True in the mask are replaced.
    - value (scalar, optional): The value to assign to elements of `array` where `mask` is True. Defaults to np.nan.

    Returns:
    - np.array: A copy of the input array with updated values where the mask is True.

    Example:
    >>> array = np.array([1, 2, 3, 4])
    >>> mask = np.array([False, True, False, True])
    >>> update_array_by_mask(array, mask, value=0)
    array([1, 0, 3, 0])
    """
    result = array.copy()
    result[mask] = value
    return result

In [12]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        # linear interpolation of NaNs
        nans, x= nan_helper(y)
        y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [13]:
def interpolate_signal(signal):
    """
    Interpolates missing values (NaNs) in a given signal array using linear interpolation.

    The function finds NaN elements in the signal, computes linear interpolation based on
    non-NaN values, and fills the NaNs with these interpolated values.

    Parameters:
        signal (numpy.ndarray): The input signal array containing NaN values that need to be interpolated.

    Returns:
        numpy.ndarray: The signal array with NaN values interpolated.
    """
    if signal is None or not isinstance(signal, np.ndarray):
        raise ValueError("Input must be a numpy ndarray.")
    
    result = signal.copy()
    nans, x = nan_helper(result)
    result[nans] = np.interp(x(nans), x(~nans), result[~nans])
    return result

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [15]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [16]:
zscore_threshold = 4
VOLTAGE_SCALING_VALUE = 0.195

In [17]:
BAND_TO_FREQ = {"theta": (4,12), "gamma": (30,51)}

In [18]:
LFP_TRACES_DF = pd.read_pickle("./proc/rce_pilot_2_01_lfp_traces_and_frames.pkl")

In [19]:
LFP_TRACES_DF.shape

(61, 23)

## Preprocessing

In [20]:
original_trace_columns = [col for col in LFP_TRACES_DF.columns if "trace" in col]

In [21]:
original_trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [22]:
for col in original_trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float32) * VOLTAGE_SCALING_VALUE)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [23]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95.354996, 82.09499, 97.5, 132.405, 123.825, ...","[46.019997, 49.335, 75.27, 97.89, 77.61, 40.55...","[61.425, 66.104996, 81.899994, 90.479996, 71.5...","[54.6, 54.405, 73.32, 86.189995, 59.085, 19.89...","[55.574997, 79.365, 128.11499, 170.43, 189.344..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95.354996, 82.09499, 97.5, 132.405, 123.825, ...","[46.019997, 49.335, 75.27, 97.89, 77.61, 40.55...","[61.425, 66.104996, 81.899994, 90.479996, 71.5...","[54.6, 54.405, 73.32, 86.189995, 59.085, 19.89...","[55.574997, 79.365, 128.11499, 170.43, 189.344..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[19.89, 29.445, 33.149998, 37.829998, 45.43499...","[29.445, 28.859999, 25.935, 23.205, 21.449999,...","[28.47, 25.349998, 22.035, 22.814999, 23.00999...","[68.64, 90.284996, 93.795, 71.564995, 90.09, 1...","[62.984997, 86.774994, 104.13, 86.96999, 75.65..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[19.89, 29.445, 33.149998, 37.829998, 45.43499...","[29.445, 28.859999, 25.935, 23.205, 21.449999,...","[28.47, 25.349998, 22.035, 22.814999, 23.00999...","[68.64, 90.284996, 93.795, 71.564995, 90.09, 1...","[62.984997, 86.774994, 104.13, 86.96999, 75.65..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 24

# Calculating modified zscore

`0.6745(xi – x̃) / MAD`

In [24]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_MAD".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: stats.median_abs_deviation(x))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [25]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_modified_zscore".format(brain_region)
    MAD_column = "{}_lfp_MAD".format(brain_region)

    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF.apply(lambda x: 0.6745 * (x[col] - np.median(x[col])) / x[MAD_column], axis=1)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [26]:
LFP_TRACES_DF[updated_column]

0     [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
1     [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
2     [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
3     [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
4     [0.27719176, 0.40500796, 0.35572946, -0.004619...
                            ...                        
56    [0.22029124, 0.7880522, 1.144606, 0.71083665, ...
57    [0.20513144, 0.031291235, -0.13791321, -0.1900...
58    [0.20513144, 0.031291235, -0.13791321, -0.1900...
59    [-1.0707098, -1.1721206, -1.1367447, -0.910339...
60    [-1.0707098, -1.1721206, -1.1367447, -0.910339...
Name: vHPC_lfp_modified_zscore, Length: 61, dtype: object

## calculating root mean sequare

In [27]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.mean(x**2))).astype(np.float32))


mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [28]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_lfp_modified_zscore,MD_lfp_modified_zscore,LH_lfp_modified_zscore,BLA_lfp_modified_zscore,vHPC_lfp_modified_zscore,mPFC_lfp_RMS,MD_lfp_RMS,LH_lfp_RMS,BLA_lfp_RMS,vHPC_lfp_RMS
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.89292616, 0.77410233, 0.9121477, 1.2249339,...","[0.47153494, 0.50638753, 0.77905774, 1.0168754...","[0.94851553, 1.0207834, 1.2646874, 1.3971784, ...","[0.66973317, 0.6673498, 0.8985389, 1.0558428, ...","[0.23408099, 0.3398509, 0.5565925, 0.7447243, ...","[0.19477782, 0.16769215, 0.19915934, 0.2704583...","[0.13429306, 0.14396672, 0.21964882, 0.2856573...","[0.22216085, 0.23908737, 0.29621443, 0.3272464...","[0.15934613, 0.15877703, 0.2139791, 0.25153923...","[0.09461661, 0.13511916, 0.21811618, 0.2901576..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.89292616, 0.77410233, 0.9121477, 1.2249339,...","[0.47153494, 0.50638753, 0.77905774, 1.0168754...","[0.94851553, 1.0207834, 1.2646874, 1.3971784, ...","[0.66973317, 0.6673498, 0.8985389, 1.0558428, ...","[0.23408099, 0.3398509, 0.5565925, 0.7447243, ...","[0.19477782, 0.16769215, 0.19915934, 0.2704583...","[0.13429306, 0.14396672, 0.21964882, 0.2856573...","[0.22216085, 0.23908737, 0.29621443, 0.3272464...","[0.15934613, 0.15877703, 0.2139791, 0.25153923...","[0.09461661, 0.13511916, 0.21811618, 0.2901576..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2918245, 0.42672452, 0.4790326, 0.5451061, ...","[0.8516992, 0.83455086, 0.7488094, 0.66878396,...","[0.84312505, 0.75009054, 0.6512414, 0.6745, 0....","[0.86968505, 1.1439322, 1.1884048, 0.9067454, ...","[0.93568933, 1.2858553, 1.5413042, 1.2887255, ...","[0.25848255, 0.38265556, 0.43080425, 0.4916236...","[0.6991664, 0.6852757, 0.6158221, 0.5509987, 0...","[0.6525567, 0.5810436, 0.50506103, 0.52293926,...","[0.83570427, 1.099236, 1.1419709, 0.8713166, 1...","[0.88643956, 1.2212558, 1.4655069, 1.2240001, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2918245, 0.42672452, 0.4790326, 0.5451061, ...","[0.8516992, 0.83455086, 0.7488094, 0.66878396,...","[0.84312505, 0.75009054, 0.6512414, 0.6745, 0....","[0.86968505, 1.1439322, 1.1884048, 0.9067454, ...","[0.93568933, 1.2858553, 1.5413042, 1.2887255, ...","[0.25848255, 0.38265556, 0.43080425, 0.4916236...","[0.6991664, 0.6852757, 0.6158221, 0.5509987, 0...","[0.6525567, 0.5810436, 0.50506103, 0.52293926,...","[0.83570427, 1.099236, 1.1419709, 0.8713166, 1...","[0.88643956, 1.2212558, 1.4655069, 1.2240001, ..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[1

## Filtering for zscore value

In [29]:
zscore_columns = [col for col in LFP_TRACES_DF.columns if "zscore" in col]

In [30]:
zscore_columns

['mPFC_lfp_modified_zscore',
 'MD_lfp_modified_zscore',
 'LH_lfp_modified_zscore',
 'BLA_lfp_modified_zscore',
 'vHPC_lfp_modified_zscore']

In [31]:
for col in zscore_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.abs(x) >= zscore_threshold)

mPFC_lfp_modified_zscore
MD_lfp_modified_zscore
LH_lfp_modified_zscore
BLA_lfp_modified_zscore
vHPC_lfp_modified_zscore


In [32]:
LFP_TRACES_DF[updated_column].head()

0    [False, False, False, False, False, False, Fal...
1    [False, False, False, False, False, False, Fal...
2    [False, False, False, False, False, False, Fal...
3    [False, False, False, False, False, False, Fal...
4    [False, False, False, False, False, False, Fal...
Name: vHPC_lfp_mask, dtype: object

In [33]:
LFP_TRACES_DF[updated_column].iloc[0].shape

(3414668,)

In [34]:
sum(LFP_TRACES_DF[updated_column].iloc[0])

120878

- Filtering raw traces by zscore

In [35]:
LFP_TRACES_DF[col].head()

0    [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
1    [0.23408099, 0.3398509, 0.5565925, 0.7447243, ...
2    [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
3    [0.93568933, 1.2858553, 1.5413042, 1.2887255, ...
4    [0.27719176, 0.40500796, 0.35572946, -0.004619...
Name: vHPC_lfp_modified_zscore, dtype: object

In [36]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_trace_filtered".format(brain_region)    
    mask_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF.apply(lambda x: update_array_by_mask(x[col], x[mask_column]), axis=1)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [37]:
LFP_TRACES_DF[col].head()

0    [55.574997, 79.365, 128.11499, 170.43, 189.344...
1    [55.574997, 79.365, 128.11499, 170.43, 189.344...
2    [62.984997, 86.774994, 104.13, 86.96999, 75.65...
3    [62.984997, 86.774994, 104.13, 86.96999, 75.65...
4    [34.32, 50.504997, 44.265, -1.365, -31.784998,...
Name: vHPC_lfp_trace, dtype: object

In [38]:
sum(np.isnan(LFP_TRACES_DF[col].iloc[0]))

0

- Calculating RMS of filtered signal

In [39]:
filtered_trace_column = [col for col in LFP_TRACES_DF if "lfp_trace_filtered" in col]

In [40]:
for col in filtered_trace_column:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS_filtered".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.nanmean(x**2))).astype(np.float32))

mPFC_lfp_trace_filtered
MD_lfp_trace_filtered
LH_lfp_trace_filtered
BLA_lfp_trace_filtered
vHPC_lfp_trace_filtered


- Dropping unnecessary columns

In [41]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "zscore" in col or "MAD" in col], errors="ignore")

In [42]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "RMS" in col and not "filtered" in col], errors="ignore")


In [43]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'mPFC_lfp_mask', 'MD_lfp_mask',
       'LH_lfp_mask', 'BLA_lfp_mask', 'vHPC_lfp_mask',
       'mPFC_lfp_trace_filtered', 'MD_lfp_trace_filtered',
       'LH_lfp_trace_filtered', 'BLA_lfp_trace_filtered',
       'vHPC_lfp_trace_filtered', 'mPFC_lfp_RMS_filtered',
       'MD_lfp_RMS_filtered', 'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered',
       'vHPC_lfp_RMS_filtered'],
      dtype='object')

- Interpolating filtered RMS

In [44]:
filtered_RMS_column = [col for col in LFP_TRACES_DF if "lfp_RMS_filtered" in col]

In [45]:
filtered_RMS_column

['mPFC_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'vHPC_lfp_RMS_filtered']

In [46]:
for col in filtered_RMS_column:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS_interpolated".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: interpolate_signal(x))

mPFC_lfp_RMS_filtered
MD_lfp_RMS_filtered
LH_lfp_RMS_filtered
BLA_lfp_RMS_filtered
vHPC_lfp_RMS_filtered


In [47]:
# RMS_columns = [col for col in LFP_TRACES_DF if "RMS" in col and "filtered" not in col]
RMS_columns = [col for col in LFP_TRACES_DF if "interpolated" in col]

In [48]:
RMS_columns

['mPFC_lfp_RMS_interpolated',
 'MD_lfp_RMS_interpolated',
 'LH_lfp_RMS_interpolated',
 'BLA_lfp_RMS_interpolated',
 'vHPC_lfp_RMS_interpolated']

In [49]:
for col in RMS_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

mPFC_lfp_RMS_interpolated
MD_lfp_RMS_interpolated
LH_lfp_RMS_interpolated
BLA_lfp_RMS_interpolated
vHPC_lfp_RMS_interpolated


In [50]:
LFP_TRACES_DF[col]

0     [0.2411, 0.3442, 0.5557, 0.7393, 0.822, 0.8115...
1     [0.2411, 0.3442, 0.5557, 0.7393, 0.822, 0.8115...
2     [0.9097, 1.253, 1.504, 1.256, 1.093, 1.31, 1.5...
3     [0.9097, 1.253, 1.504, 1.256, 1.093, 1.31, 1.5...
4     [0.2634, 0.3875, 0.3396, -0.010475, -0.2439, -...
                            ...                        
56    [0.221, 0.79, 1.147, 0.713, 0.3667, 0.344, 0.3...
57    [0.1772, 0.01003, -0.1527, -0.2029, -0.3599, -...
58    [0.1772, 0.01003, -0.1527, -0.2029, -0.3599, -...
59    [-1.064, -1.166, -1.131, -0.905, -0.797, -0.73...
60    [-1.064, -1.166, -1.131, -0.905, -0.797, -0.73...
Name: vHPC_lfp_RMS_interpolated, Length: 61, dtype: object

In [51]:
for col in original_trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [52]:
LFP_TRACES_DF[col]

0     [55.56, 79.4, 128.1, 170.4, 189.4, 187.0, 182....
1     [55.56, 79.4, 128.1, 170.4, 189.4, 187.0, 182....
2     [63.0, 86.75, 104.1, 87.0, 75.7, 90.7, 110.56,...
3     [63.0, 86.75, 104.1, 87.0, 75.7, 90.7, 110.56,...
4     [34.3, 50.5, 44.25, -1.365, -31.78, -28.28, -6...
                            ...                        
56    [18.92, 67.7, 98.25, 61.03, 31.39, 29.44, 33.3...
57    [31.0, 1.755, -26.72, -35.5, -63.0, -95.2, -11...
58    [31.0, 1.755, -26.72, -35.5, -63.0, -95.2, -11...
59    [-88.3, -96.75, -93.8, -75.06, -66.1, -61.03, ...
60    [-88.3, -96.75, -93.8, -75.06, -66.1, -61.03, ...
Name: vHPC_lfp_trace, Length: 61, dtype: object

In [53]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "filtered" in col], errors="ignore")

In [54]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'mPFC_lfp_mask', 'MD_lfp_mask',
       'LH_lfp_mask', 'BLA_lfp_mask', 'vHPC_lfp_mask',
       'mPFC_lfp_RMS_interpolated', 'MD_lfp_RMS_interpolated',
       'LH_lfp_RMS_interpolated', 'BLA_lfp_RMS_interpolated',
       'vHPC_lfp_RMS_interpolated'],
      dtype='object')

# Calculating the power with CWT

In [57]:
RMS_columns

['mPFC_lfp_RMS_interpolated',
 'MD_lfp_RMS_interpolated',
 'LH_lfp_RMS_interpolated',
 'BLA_lfp_RMS_interpolated',
 'vHPC_lfp_RMS_interpolated']

In [117]:
#Initialize
fs = 1000

f0 = 0.5 #lowest frequency
# f1 = 51.5 #highest frequency
# fn = 51 #number of frequencies
f1 = 50 #highest frequency
fn = 50 #number of frequencies
nthreads = 14


In [97]:
signal = LFP_TRACES_DF["LH_lfp_RMS_interpolated"].iloc[0][:10000]

# Calculate CWT without plotting...


In [119]:
freqs, out = fcwt.cwt(signal, fs, f0, f1, fn, nthreads=nthreads, norm=True)

In [114]:
RMS_columns

['mPFC_lfp_RMS_interpolated',
 'MD_lfp_RMS_interpolated',
 'LH_lfp_RMS_interpolated',
 'BLA_lfp_RMS_interpolated',
 'vHPC_lfp_RMS_interpolated']

In [121]:
for col in RMS_columns:
    # brain_region = col.split("_")[0]
    brain_region = col.replace("_lfp", "")
    print(brain_region)

    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[col].apply(lambda x: fcwt.cwt(x, fs, f0, f1, fn, nthreads=nthreads, norm=True)[1])
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[col].apply(lambda x: fcwt.cwt(x[:10000], fs, f0, f1, fn, nthreads=nthreads, norm=True)[0])
   
    except Exception as e: 
        print(e)

mPFC_RMS_interpolated


KeyboardInterrupt: 

In [ ]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_CWT.pkl")

In [55]:
raise ValueError()

ValueError: 

## Calculating phase of signals

In [ ]:
from scipy.signal import butter, filtfilt, hilbert

- Filtering for theta and gamma

In [ ]:
fs = 1000
order=4

In [ ]:
freq_band = [4, 12]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [ ]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_theta_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float16))

In [ ]:
LFP_TRACES_DF[updated_column]

In [ ]:
freq_band = [30, 50]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [ ]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_gamma_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float16))

- Calculating the phase

In [ ]:
band_columns = [col for col in LFP_TRACES_DF if "band" in col]

In [ ]:
band_columns

In [ ]:
for col in band_columns:
    print(col)
    brain_region = col.replace("_band", "")
    updated_column = "{}_phase".format(brain_region)
    print(updated_column)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.angle(hilbert(x), deg=False).astype(np.float16))

In [ ]:
LFP_TRACES_DF.columns

# Getting all the pairs

In [ ]:
input_columns = [col for col in LFP_TRACES_DF.columns if "interpolated" in col]

In [ ]:
input_columns

In [ ]:
all_suffixes = set(["_".join(col.split("_")[1:]) for col in input_columns])

In [ ]:
all_suffixes

In [ ]:
brain_region_pairs = generate_pairs(sorted(list(set([col.split("lfp")[0].strip("_") for col in sorted(input_columns)]))))

In [ ]:
brain_region_pairs

In [ ]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
        print(region_1)
        print(region_2)

# Calculate Granger's

In [ ]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        suffix_for_name = suffix.replace("lfp", "").strip("_")
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
         # Define base name for pair
        pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}_{suffix_for_name}"
        print(pair_base_name)

In [ ]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
        region_1_base_name = region_1.split('_')[0]
        region_2_base_name = region_2.split('_')[0]
        pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
        print(pair_base_name)

        try:
            # Define column names
            multitaper_col = f"{pair_base_name}_granger_multitaper"
            connectivity_col = f"{pair_base_name}_granger_connectivity"
            frequencies_col = f"{pair_base_name}_granger_frequencies"
            granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
            granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"

            # Apply Multitaper function
            LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
                lambda x: Multitaper(
                    time_series=np.array([x[region_1], x[region_2]]).T, 
                    sampling_frequency=RESAMPLE_RATE, 
                    time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                    time_window_step=TIME_WINDOW_STEP, 
                    time_window_duration=TIME_WINDOW_DURATION
                ), 
                axis=1
            )

            # Apply Connectivity function
            LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
                lambda x: Connectivity.from_multitaper(x)
            )

            # Apply frequencies and granger functions
            LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
                lambda x: x.frequencies[:62]
            )

            LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[connectivity_col].apply(
                lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1]
            )

            LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[connectivity_col].apply(
                lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0]
            )

            LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[granger_1_2_col].apply(lambda x: x.astype(np.float32)[:, :62])
            LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[granger_2_1_col].apply(lambda x: x.astype(np.float32)[:, :62])

        except Exception as e: 
            print(e)

        # Drop temporary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
        


- Getting the timestamps of the granger

In [ ]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [ ]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")

## Coherece Calculation

- Calculating the coherence

In [ ]:
for first_region, second_region in brain_region_pairs:
    for suffix in all_suffixes:
        suffix_for_name = suffix.replace("lfp", "").strip("_")
        region_1 = "_".join([first_region, suffix])
        region_2 = "_".join([second_region, suffix])
         # Define base name for pair
        pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}_{suffix_for_name}"
        print(region_1)
        print(region_2)
        print(pair_base_name)

        try:
            # Define column names
            multitaper_col = f"{pair_base_name}_coherence_multitaper"
            connectivity_col = f"{pair_base_name}_coherence_connectivity"
            frequencies_col = f"{pair_base_name}_coherence_frequencies"
            coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

            # Apply Multitaper function
            LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
                lambda x: Multitaper(
                    time_series=np.array([x[region_1], x[region_2]]).T, 
                    sampling_frequency=RESAMPLE_RATE, 
                    time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                    time_window_step=TIME_WINDOW_STEP, 
                    time_window_duration=TIME_WINDOW_DURATION
                ), 
                axis=1
            )

            # Apply Connectivity function
            LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
                lambda x: Connectivity.from_multitaper(x)
            )

            # Apply frequencies and coherence functions
            LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
                lambda x: x.frequencies[:62]
            )
            LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[connectivity_col].apply(
                lambda x: x.coherence_magnitude()[:,:,0,1]
            )

            LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[coherence_col].apply(lambda x: x[:,:62].astype(np.float32))

        except Exception as e: 
            print(e)

        # Drop temporary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

- Getting the timestamps of the coherence

In [ ]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [ ]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_coherence.pkl")

In [ ]:
raise ValueError()

# Power Calcuation

- Getting the column name of all the traces

In [ ]:
input_columns = [col for col in LFP_TRACES_DF.columns if "trace" in col or "RMS" in col]

In [ ]:
input_columns

In [ ]:
for col in input_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

- Calcuating the power at each frequency band

In [ ]:
LFP_TRACES_DF[col].iloc[0]

In [ ]:
for col in input_columns:
    # brain_region = col.split("_")[0]
    brain_region = col.replace("_lfp", "")
    print(brain_region)

    # Define column names
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float16))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

- Getting the timestamps of the power

In [ ]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [ ]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF["mPFC_RMS_filtered_power_all_frequencies_all_windows"].head()

In [ ]:
LFP_TRACES_DF["mPFC_RMS_filtered_power_all_frequencies_all_windows"].iloc[4].shape

In [ ]:
LFP_TRACES_DF["mPFC_lfp_RMS_filtered"].head()

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_trace"].iloc[0])

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_RMS"].iloc[0])

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_RMS"].iloc[0])

In [ ]:
LFP_TRACES_DF["BLA_"].iloc[0]

In [ ]:
raise ValueError()

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_RMS_filtered"].iloc[0])

In [ ]:
LFP_TRACES_DF["BLA_trace_power_all_frequencies_all_windows"].apply(lambda x: np.sum(np.isnan(x[:,3:13])))

In [ ]:
LFP_TRACES_DF["BLA_RMS_filtered_power_all_frequencies_all_windows"].apply(lambda x: np.sum(np.isnan(x[:,3:13])))

In [ ]:
print("hello")

In [ ]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

## Calculating the averages

In [ ]:
LFP_TRACES_DF.columns

In [ ]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

In [ ]:
raise ValueError()